### Just for test

[original source code](https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2006%20-%20Federated%20Learning%20on%20MNIST%20using%20a%20CNN.ipynb)

## Imports and model specifications

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
import syft as sy  # <-- NEW: import the Pysyft library
hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

W0805 15:05:37.647576  5316 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'd:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0805 15:05:37.662046  5316 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [3]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 2
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

# device = torch.device("cuda" if use_cuda else "cpu")
device = torch.device('cpu')
print(device)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print(kwargs)

cpu
{'num_workers': 1, 'pin_memory': True}


## Data loading and sending to workers

In [4]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- NEW: we distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

W0805 15:05:52.459892  5316 dataloader.py:197] The following options are not supported: num_workers: 1, pin_memory: True


## CNN specification

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [6]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    print(device)
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [7]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Launch the training !

In [8]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "./models/mnist_cnn.pt")

cpu
Train Epoch: 1 [0/60032 (0%)]	Loss: 2.305134
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.156802
Train Epoch: 1 [3840/60032 (6%)]	Loss: 1.896594
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.440288
Train Epoch: 1 [7680/60032 (13%)]	Loss: 0.867165
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.654450
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.593086
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.455329
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.370902
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.304455
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.313768
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.369191
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.237834
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.188108
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.522550
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.224813
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.143817
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.267367
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.187140
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.303518
Train Ep

PicklingError: Can't pickle <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>: attribute lookup Tensor on syft.frameworks.torch.tensors.interpreters.native failed